In [1]:
import os, re, json, shutil, time
import numpy as np
from ase.io import read, write, iread
from ase.io.trajectory import Trajectory
from ase.visualize import view, external
from ase import units
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.geometry.analysis import Analysis

from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin

from ase.calculators.cp2k import CP2K
from ase.calculators.orca import ORCA
from ase.calculators.psi4 import Psi4

from IPython.utils.io import capture_output
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.stats import norm
from tqdm import tqdm

### Function definitions

In [2]:
def center_COM(atoms):
    """Center the COM of `atoms` in their own cell."""

    # position of box center
    c = 0.5 * atoms.get_cell().sum(axis=0) + atoms.get_celldisp().T

    # center of mass shifted to box center
    positions = atoms.get_positions() - atoms.get_center_of_mass() + c

    # update positions in place
    atoms.set_positions(positions)


def new_sim_dir(path="", prefix="simulation"):
    '''
    Looks for all non-empty enumerated directories of desired format and returns new dirname with id one greater.
    '''
    reg_dir_num = re.compile("_[0-9]{3}_")
    id_list = [int(re.findall(reg_dir_num, f.path)[0].replace('_','')) for f in os.scandir() 
               if f.is_dir() and "/simulation_" in f.path and len(os.listdir(f)) > 0]
    current = max([0] + id_list)
    
    new_dirname = "{}_{:03d}_".format(prefix, current + 1)
    print(new_dirname)
    
    return new_dirname    


def save_parameters(pars, filename="parameters.json"):
    par_rev = pars.copy()
    par_rev['atoms'] = par_rev['atoms'].get_chemical_symbols()
    
    with open(par_rev['output_dir'] + filename, 'w') as jsfile:
        json.dump(par_rev, jsfile, indent=4)


def load_energies(path):
    reg_num = re.compile("[-0-9.]+")

    times = list()
    E_tot = list()
    E_pot = list()
    E_kin = list()

    with open(path, 'r') as file:
        print("Header: {}".format(next(file)))        # dumps the header
        for line in file:
            num_data = re.findall(reg_num, line.replace("\n",""))
            for i, lst in enumerate([times, E_tot, E_pot, E_kin]):
                lst.append(float(num_data[i]))
                
    return times, E_tot, E_pot, E_kin


class MyTimer:
    def __init__(self, output_path):
        self.path = output_path
        self.data = list()
        self.stack = dict()
        
    def start(self,label):
        t0 = time.time()
        self.stack.update({label:t0})
        
    def stop(self,label,save_data=False):
        dt = time.time() - self.stack.pop(label)
        self.data.append([label, dt])
        
        if save_data is True:
            self.save_data()
            
    def stop_start(self, label='step'):
        dt = time.time() - self.stack.pop(label) if label in self.stack.keys() else 0
        self.data.append([label, dt]) if dt != 0 else None
        
        t1 = time.time()
        self.stack.update({label:t1})
    
        
    def save_data(self):
        with open(self.path, 'w') as file:
            file.write("Date and time of the simulation run:   {} \n".format(time.strftime("%A, %d %b %Y %H:%M:%S", time.gmtime()) ))
            file.write(" LABEL   TIME [s] \n")
            
            for line in self.data:
                file.write(" {}   {:.3f} s \n".format(line[0], float(line[1])))
                
                
def transform_frames(path, n_sim, ox_id=[0,3], H_id=4):
    def center_atom_pair(frame, pos1, pos2, center):
        d_center = center - (pos1 + (pos2 - pos1)/2)
        frame.translate(d_center)
        
    for i in range(n_sim):
        new_traj = Trajectory(path + 'coords_transformed_{:03d}.traj'.format(i), 'w')

        times, E_tot, E_pot, E_kin = load_energies(path + 'energies_{:03d}.log'.format(i))

        ird = iread(path + "coords_{:03d}.traj".format(i), index=':')

        for frame in ird:
            cell_center = frame.get_cell().sum(axis=0) / 2
            pos1, pos2 = frame.get_positions()[ox_id]

            center_atom_pair(frame, pos1, pos2, cell_center)
            frame.rotate(pos2 - pos1, [1,0,0], center=cell_center )   # orient oxygen pair in x-axis direction

            shared_H_pos = frame.get_positions()[H_id]
            frame.rotate((shared_H_pos - cell_center)*np.array([0,1,1]), [0,0,1], center=cell_center)   # restrict shared H movement to x-z  plane only  

            new_traj.write(frame)

        write(path + "coords_transformed_{:03d}.xyz".format(i), read(path + "coords_transformed_{:03d}.traj".format(i), index=':'), format="xyz")


# **Simulation**

## **Define initial structure**

In [3]:
zundel = read("./input_files/initial.pdb",format='proteindatabank')

In [4]:
L = 12.0 * np.ones(3)
zundel.set_cell(L)
center_COM(zundel)

print(zundel.get_positions())
print(zundel.get_center_of_mass())

[[5.84488069 6.76080447 6.90022558]
 [5.67888069 7.70080447 6.75722558]
 [5.85988069 6.56480447 7.84522558]
 [6.15188069 5.26080447 5.07122558]
 [6.01188069 5.93680447 5.96622558]
 [7.02988069 4.86680447 4.95822558]
 [5.47088069 4.58780447 4.92622558]]
[6. 6. 6.]


In [5]:
#view(zundel, viewer='vmd')

## **Calculator setup**

### Setup CP2K

In [6]:
os.environ['CP2K_DATA_DIR'] = "/home/mptacek/md_project"
CP2K.command = '/home/mptacek/software/cp2k_shell.ssmp --shell' 

In [7]:
# load CP2K input file
with open("./input_files/zundel_cp2k.inp") as f_inp:
    inp_cp2k = f_inp.read()

In [8]:
calculator = CP2K(
    inp=inp_cp2k,
    directory='calculator',
    stress_tensor=False,
    cutoff=None,
    basis_set_file=None,
    potential_file=None,
    poisson_solver=None,
    max_scf=None,
    xc=None,
    basis_set=None,
    pseudo_potential=None,
)
pp = 'CP2K'

### Setup ORCA

In [6]:
ORCA.command = '/home/mptacek/software/orca_5_0_2_linux_x86-64_openmpi411/orca PREFIX.inp > PREFIX.out'

orca_pars = dict(label='orcacalc',
                 charge=1,
                 mult=1,
                 #orcasimpleinput='HF cc-PVTZ',
                 orcasimpleinput='B3LYP cc-PVTZ',
                 orcablocks='%maxcore 2000 %pal nprocs 4 end',
                )

pp = orca_pars

calculator = ORCA(label=pp['label'], charge=pp['charge'], mult=pp['mult'], orcasimpleinput=pp['orcasimpleinput'],
                  orcablocks=pp['orcablocks'],
                 )

### Setup Psi4

In [6]:
psi4_pars = dict(method='b3lyp',
                 #method='hf',
                 #method='m06',
                 #basis='6-311g_d_p_',
                 #basis='aug-cc-pVTZ',
                 basis='def2-TZVP',
                 charge=1,
                 multiplicity=1,
                 memory='6GB',
                 num_threads=4,
                 label='psi4-calc',
                )

pp = psi4_pars
calculator = Psi4(atoms=zundel, method=pp['method'], basis=pp['basis'], charge=pp['charge'],
                  multiplicity=pp['multiplicity'], memory=pp['memory'], num_threads=pp['num_threads'],
                  label=pp['label'],
                 )


  Memory set to   5.588 GiB by Python driver.
  Threads set to 4 by Python driver.


## **Simulation setup** - single

### Spawn new zundel

In [9]:
zun = zundel.copy()

new_output_dir = new_sim_dir()

simulation_029_


In [10]:
zun.set_pbc(False)
zun

Atoms(symbols='OH2OH3', pbc=False, cell=[12.0, 12.0, 12.0], atomtypes=..., bfactor=..., occupancy=..., residuenames=..., residuenumbers=...)

In [11]:
parameters = dict(atoms=zun,             # atoms
                  timestep=1*units.fs,   # timestep
                  temperature=300,       # temp in K 
                  sim_length=5000*units.fs,
                  friction=0.002,
                  logfile='energies.log', #'-',
                  trajectory=None,
                  fixcm=True,
                  loginterval=1,
                  output_dir='./{}{}/'.format(new_output_dir, 'xxx'),
                  nvt=True,
                  calc_pars=pp,
                 )

p = parameters

### Set calculator + SP energy

In [12]:
# set calculator
p['atoms'].set_calculator(calculator)

# set and create output directory
calc_name = p['atoms'].get_calculator().name
p['output_dir'] = './{}{}/'.format(new_output_dir, calc_name)
print("Output directory is set to: {} \n".format(p['output_dir']))
Path(p['output_dir']).mkdir(parents=False, exist_ok=True)

timer = MyTimer(p['output_dir'] + "timings.txt")

# test - get total potential energy:
print("E_pot = {:.4f}".format(p['atoms'].get_potential_energy()))

Output directory is set to: ./simulation_015_cp2k/ 

E_pot = -224.4287


### Thermostats

In [13]:
if p['nvt'] is True:
    moldyn = Langevin(p['atoms'],               # atoms
                  timestep=p['timestep'],       # timestep
                  temperature_K=p['temperature'],     # temp in K 
                  friction=p['friction'],
                  logfile=p['output_dir'] + p['logfile'],
                  trajectory=p['trajectory'],
                  fixcm=p['fixcm'],
                  loginterval=p['loginterval'],
                 )
elif p['nvt'] is False:
    moldyn = VelocityVerlet(p['atoms'],                  # atoms
                        timestep=p['timestep'],          # timestep
                        logfile=p['output_dir'] + p['logfile'],
                        trajectory=p['trajectory'],
                        loginterval=p['loginterval'],
                       )
else:
    ValueError()

### Save parameter files

In [15]:
MaxwellBoltzmannDistribution(p['atoms'], temperature_K=p['temperature'])

In [16]:
interval = p['loginterval']

traj = Trajectory(p['output_dir'] + 'coords.traj', 'w', p['atoms'])

# save parameter dictionary as JSON
save_parameters(p)

if calc_name == 'cp2k':
    shutil.copy("./input_files/zundel_cp2k.inp","./"+p['output_dir']+"/zundel_cp2k.inp")

### **Initialize and run the simulation**

In [17]:
moldyn.attach(traj.write, interval=interval)
moldyn.attach(timer.stop_start, interval=interval)

In [18]:
timestep = p['timestep']
duration = p['sim_length']
n_steps = round(duration / timestep)
print("Simulation timing:    {}  x  {} fs = {} fs".format(int(timestep / units.fs), int(n_steps), int(duration / units.fs)))

Simulation timing:    1  x  4000 fs = 4000 fs


In [21]:
with capture_output() as captured:
    timer.start('simulation')
    printenergy()
    moldyn.run(n_steps)
    write(p['output_dir']+"coords.xyz", read(p['output_dir'] + "coords.traj",index=':'), format="xyz")
    timer.stop('simulation', save_data=True)

In [33]:
#write("coords.xyz", read("coords.traj",index=':'), format="xyz")

## **Simulation setup and run** - multi copy

In [9]:
zun = zundel.copy()

new_output_dir = new_sim_dir()

simulation_049_


In [10]:
parameters = dict(atoms=zun,             # atoms
                  timestep=0.5 *units.fs,  # timestep
                  temperature=77,       # temp in K 
                  sim_length=5000 *units.fs,
                  friction=0.002,
                  logfile='energies.log', #'-',
                  trajectory=None,
                  fixcm=True,
                  loginterval=2,
                  output_dir='./{}{}/'.format(new_output_dir, 'xxx'),
                  nvt=True,
                  repeat=1,
                  calc_pars=pp,
                 )
p = parameters

In [11]:
# set and create output directory
calc_name = calculator.name
print("Calculator name: "+calc_name)

p['output_dir'] = './{}{}/'.format(new_output_dir, calc_name)
print("Output directory is set to: {} \n".format(p['output_dir']))
Path(p['output_dir']).mkdir(parents=False, exist_ok=True)

if calc_name == 'cp2k':
    shutil.copy("./input_files/zundel_cp2k.inp","./"+p['output_dir']+"/zundel_cp2k.inp")

timer = MyTimer(p['output_dir'] + "timings.txt")
    
# save parameter dictionary as JSON
save_parameters(p)

interval = p['loginterval']
timestep = p['timestep']
duration = p['sim_length']
n_steps = round(duration / timestep)
print("Simulation timing:    {}  x  {} fs = {} fs".format(int(n_steps), int(timestep / units.fs), int(duration / units.fs)))

Calculator name: cp2k
Output directory is set to: ./simulation_049_cp2k/ 

Simulation timing:    10000  x  0 fs = 5000 fs


In [12]:
timer.start('simulation')
for i_sim in tqdm(range(p['repeat'])):
    timer.start('iter_{}'.format(i_sim))
    zuni = zundel.copy()
    
    # set calculator
    zuni.set_calculator(calculator)
    
    p['atoms'] = zuni
    
    if p['nvt'] is True:
        moldyni = Langevin(zuni, timestep=p['timestep'], temperature_K=p['temperature'], friction=p['friction'], 
                          logfile=p['output_dir'] + p['logfile'].replace(".log","_{:03d}.log".format(i_sim)), 
                          trajectory=p['trajectory'], fixcm=p['fixcm'], loginterval=p['loginterval'],
                         )
    elif p['nvt'] is False:
        moldyni = VelocityVerlet(zuni, timestep=p['timestep'], trajectory=p['trajectory'], loginterval=p['loginterval'],
                                logfile=p['output_dir'] + p['logfile'].replace(".log","_{:03d}.log".format(i_sim)),
                               )
    else:
        ValueError()

    traj = Trajectory(p['output_dir'] + 'coords.traj'.replace(".traj","_{:03d}.traj".format(i_sim)), 'w', zuni)

    MaxwellBoltzmannDistribution(zuni, temperature_K=p['temperature'])
    
    moldyni.attach(traj.write, interval=interval)
    moldyni.attach(timer.stop_start, interval=interval)

    with capture_output() as captured:
        moldyni.run(n_steps)
        write(p['output_dir']+"coords.xyz".replace(".xyz","_{:03d}.xyz".format(i_sim)), 
              read(p['output_dir'] + "coords.traj".replace(".traj","_{:03d}.traj".format(i_sim)),index=':'), format="xyz")
        
    timer.stop('iter_{}'.format(i_sim))
        
timer.stop('simulation', save_data=True)
transform_frames(p['output_dir'], p['repeat'])

100%|███████████████████████████████████████████████████████████████████████████████████████| 1/1 [04:43<00:00, 283.34s/it]


Header: Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]



In [11]:
## save times in case of error in above cell !!! 
#timer.stop('simulation', save_data=True)